In [1]:
function img = data2img( data, r, c )

  data = data';
  [d,n] = size( data );
  bs = sqrt( d );
  if( r*c != d*n )
    disp( "Size mismatch" );
    return;
  end
  
  img = zeros( r, c );
  count = 1;
  for i=1:bs:r
    for j=1:bs:c
      img(i:i+bs-1,j:j+bs-1) = reshape( data(:,count), bs,bs );
      count = count + 1;
    end
  end
end

In [2]:
function ret = img2data( img, bs )

  [r,c] = size( img );

  ret = [];
  for i=1:bs:r
    for j=1:bs:c
      ret = [ret reshape( img(i:i+bs-1,j:j+bs-1), bs*bs, 1 ) ];
    end
  end

  ret = ret';
  
end

In [3]:
function [P, L] = EIG(A)
    % 正方行列かどうかチェック
    [r, c] = size(A);
    if (r != c)
        disp("Not a square matrix.");
        return;
    end
    
    % 対称行列かどうかチェック
    diff = A - A';
    if (trace(diff' * diff) / trace(A' * A) > 0.0000000000001)
        disp("Not a symmetric matrix.");
        return;
    end
    
    % 念の為厳密に対称行列にしておく
    L = (A + A') / 2;
    P = eye(r);
    
    while(1)
        % 非対角要素で絶対値が最大のものを探す
        max_val = 0;
        max_i = 1;
        max_j = 1;
        for i = 1:r
            % 絶対値が最大となる位置を探す
            % それぞれ max_i と max_j に格納
            for k = 1:r
                for l = 1:r
                    if (k != l)
                        if (max_val < abs(L(k, l)))
                            max_val = abs(L(k, l));
                            max_i = k;
                            max_j = l;
                        end
                    end
                end
            end
        end


        if (max_val < 0.0000000000001)
            break;
        end
        
        % 角度 θ と回転行列 S を算出
        theta = atan2(2 * L(max_i, max_j), L(max_i, max_i) - L(max_j, max_j)) / 2;
        S = eye(r, r);
        S(max_i, max_i) = cos(theta);
        S(max_i, max_j) = -sin(theta);
        S(max_j, max_i) = sin(theta);
        S(max_j, max_j) = cos(theta);

        % L と P を更新
        L = S' * L * S;
        P = P * S;
    end
    
    % 固有値の絶対値の大きい順にソート
     [B, I] = sort(diag(L), "descend");
     L = diag(B);
     P = P(:, I);
     return;
end

In [4]:
function [P, c, a] = PCA(data)
    % データサイズ取得
    [n, d] = size(data);
    % 分散共分散行列算出
    cov_mat = cov(data);
    
    % スペクトル分解
    [P, L] = EIG(cov_mat);

    % 寄与率算出
    c = zeros(d, 1);
    for i = 1:d
        c(i) = L(i, i) / trace(L);
    end

    % 累積寄与率算出
    a = zeros(d, 1);
    for i = 1:d
        for k = 1:i
            a(i) += c(k);
        end
    end
end

In [5]:
function [ret, num] = PCA_represent(data, ac)
    % 主成分分析
    [P, c, a] = PCA(data);
    
    % 平均ベクトル算出
    ave = mean(data);

    % 後で全データに平均をまとめて足しこむための準備
    avemat = ones(size(data, 1), 1) * ave;
    
    % 本数を取得
    num = size(find(a < ac), 1);

    % 累積寄与率 ac となる部分の主成分を取得
    basis = P(:, 1:num);

    % 射影子を作成
    Prj = basis * basis';

    % データ再構成
    ret = (data - avemat) * Prj' + avemat;
    return;
end

In [7]:
# 画像読み込み
img = im2double(imread("sample/LENA256.pgm"));
# 画像を多変量データに変換
data = img2data(img, 8);

In [10]:
[n, d] = size(data)

n =  1024
d =  64


In [11]:
cov_mat = cov(data);

In [12]:
size(cov_mat)

ans =

   64   64



In [16]:
[P, L] = eig(cov_mat);
[B, I] = sort(diag(L), "descend");
P = P(:, I);
L = diag(B);

In [19]:
c = zeros(d, 1);
for i = 1:d
    c(i) = L(i, i) / trace(L);
end

In [21]:
% 累積寄与率算出
a = zeros(d, 1);
for i = 1:d
    for k = 1:i
        a(i) += c(k);
    end
end

In [22]:
a

a =

   0.81335
   0.88511
   0.91618
   0.93690
   0.95000
   0.95954
   0.96474
   0.96984
   0.97386
   0.97711
   0.97985
   0.98196
   0.98379
   0.98524
   0.98657
   0.98769
   0.98876
   0.98972
   0.99061
   0.99130
   0.99198
   0.99259
   0.99314
   0.99362
   0.99406
   0.99445
   0.99482
   0.99516
   0.99548
   0.99579
   0.99606
   0.99631
   0.99655
   0.99677
   0.99698
   0.99718
   0.99736
   0.99754
   0.99770
   0.99785
   0.99800
   0.99813
   0.99826
   0.99839
   0.99851
   0.99862
   0.99873
   0.99883
   0.99894
   0.99903
   0.99912
   0.99921
   0.99930
   0.99938
   0.99945
   0.99952
   0.99959
   0.99966
   0.99972
   0.99979
   0.99984
   0.99990
   0.99995
   1.00000

